In [1]:
import ee
import geemap
from geemap import ml
from sklearn import ensemble
import pandas as pd


ee.Authenticate()
ee.Initialize(project='ee-facuboladerasgee')

In [15]:
roi = ee.FeatureCollection('projects/ee-facuboladerasgee/assets/costa_uruguay')
Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap('SATELLITE')
Map.centerObject(roi)

year = 2021

start = f'{year}-01-01'
end = f'{year}-12-31'

No such comm: 8e0a136934814a0eb4ef2c3b62c0da9b
No such comm: 8e0a136934814a0eb4ef2c3b62c0da9b
No such comm: 8e0a136934814a0eb4ef2c3b62c0da9b
No such comm: 8e0a136934814a0eb4ef2c3b62c0da9b


In [24]:
s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .filterDate(start, end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

cloud_col = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
    .filterBounds(roi) \
    .filterDate(start, end)

def mask_clouds(image):
    cloud_score = ee.Image(cloud_col.filterMetadata('system:index', 'equals', image.get('system:index')).first())
    mask = cloud_score.select('probability').lt(10)
    return image.updateMask(mask)

s2_masked = s2_col.map(mask_clouds)
s2_image = s2_masked.median().toFloat().clip(roi)

ndvi = s2_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
mndwi = s2_image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
ndbi = s2_image.normalizedDifference(['B11', 'B8']).rename('NDBI')

evi = s2_image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4'),
        'BLUE': s2_image.select('B2')
    }
).rename('EVI')

savi = s2_image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
    {
        'NIR': s2_image.select('B8'),
        'RED': s2_image.select('B4')
    }
).rename('SAVI')

# Add calculated indices to Sentinel-2 image
s2_image = s2_image.addBands([ndvi, mndwi, ndbi, evi, savi])

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(roi) \
    .filterDate(start, end)

dw_label = dw_col.select('label').median().clip(roi)
s2_image = s2_image.addBands(dw_label.rename('label'))

dem = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
slope = ee.Terrain.slope(dem).rename('slope').clip(roi)
aspect = ee.Terrain.aspect(dem).rename('aspect').clip(roi)

s2_image = s2_image.addBands([dem.rename('elevation'), slope, aspect])

In [25]:
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

def create_filtered_collection(polarization):
    return ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
        .select(polarization) \
        .map(mask_edge)

img_vv = create_filtered_collection('VV')
img_vh = create_filtered_collection('VH')

s1_vv = img_vv.median().rename('VV').clip(roi)
s1_vh = img_vh.median().rename('VH').clip(roi)

s1_combined = ee.Image.cat([s1_vv, s1_vh])

rvi = s1_combined.expression(
    '4 * VH / (VV + VH)',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('RVI')

vv_plus_vh = s1_combined.expression(
    'VV + VH',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('VV_plus_VH')

vv_minus_vh = s1_combined.expression(
    'VV - VH',
    {
        'VV': s1_combined.select('VV'),
        'VH': s1_combined.select('VH')
    }
).rename('VV_minus_VH')

s1_image = s1_combined.addBands([rvi, vv_plus_vh, vv_minus_vh])
image = s2_image.addBands([s1_image, dem.rename('elevation'), slope])

In [26]:
def calculate_texture_metrics(image, band_name, roi, size=5):
    
    band_int = image.select(band_name).toInt32().clip(roi)
    
    # Calcular texturas usando la banda convertida a int32
    glcm = band_int.glcmTexture(size=size)
    
    contrast = glcm.select(f'{band_name}_contrast').rename(f'{band_name}_Contrast')
    correlation = glcm.select(f'{band_name}_corr').rename(f'{band_name}_Correlation')
    entropy = glcm.select(f'{band_name}_ent').rename(f'{band_name}_Entropy')
    inertia = glcm.select(f'{band_name}_inertia').rename(f'{band_name}_Inertia')
    
    return image.addBands([contrast, correlation, entropy, inertia])



image = calculate_texture_metrics(image, 'VV', roi)
image = calculate_texture_metrics(image, 'VH', roi)

bands = ['B11', 'B12', 'B8', 'B2', 'B3', 'B4', 'VV', 'VH', 'RVI', 'VV_plus_VH', 'VV_minus_VH', 'elevation', 'slope','aspect','label', 'EVI','SAVI','NDVI', 'MNDWI', 'NDBI','VV_Contrast', 'VV_Correlation', 'VV_Entropy' ,'VV_Inertia' ,'VH_Contrast' ,'VH_Correlation' ,'VH_Entropy' ,'VH_Inertia']
image = image.select(bands)

In [27]:
# ch_image = ee.Image("projects/ee-facuboladerasgee/assets/CHM_pampa")
# ch_b1 = ch_image.select("b1").rename("CH")

# image = image.addBands(ch_b1)

Map.centerObject(roi, 10)
Map.addLayer(image, {}, 'b1 Band', False)
Map

Map(bottom=620973.0, center=[-31.471920141880172, -58.15676037656757], controls=(WidgetControl(options=['posit…

In [28]:
def mask_based_on_label_values(image, values):
    
    dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .select('label') \
        .median() \
        .clip(roi)
    
    # Crear una máscara basada en los valores de la lista
    label_mask = dynamic_world.eq(values[0])  # Inicializar con el primer valor
    for value in values[1:]:
        label_mask = label_mask.Or(dynamic_world.eq(value))  # Añadir los demás valores
    
    return image.updateMask(label_mask)

values_to_mask = [0,1,2,3,4,5,6,7,8] 


def get_gedi_data(band_name):
    return ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY') \
        .filterBounds(roi) \
        .filterDate(start, end) \
        .map(lambda image: image.updateMask(
            image.select('degrade_flag').eq(0)
            .And(image.select('l2_quality_flag').eq(1))
            .And(image.select('l4_quality_flag').eq(1)))) \
        .select(band_name) \
        .median() \
        .toFloat() \
        .clip(roi)

# Obtener las bandas agbd, agbd_se y l4_quality_flag
gediData_agbd = get_gedi_data('agbd').rename('agbd')
gediData_agbd_se = get_gedi_data('agbd_se').rename('agbd_se')
gedi_combined = ee.Image.cat([gediData_agbd, gediData_agbd_se])
gedi_masked = mask_based_on_label_values(gedi_combined, values_to_mask)

def convert_to_float(image):
    return image.float()

gedi_masked = convert_to_float(gedi_masked)
image = image.addBands(gedi_masked)

In [29]:
import time

sample = image.sample(
    scale=30,  
    region=roi,
    geometries=True 
)

export_task = ee.batch.Export.table.toDrive(
    collection=sample,
    description='ExportSampleToCSV',
    folder='EE_costa_completa',
    fileNamePrefix=f'Datos_RF_2021',
    fileFormat='CSV'
)

export_task.start()
export_task.status()

while export_task.active():
    print('Exportación en progreso...')
    time.sleep(30)  
    
    if export_task.status()['state'] == 'COMPLETED':
        print(f'Exportación completada con éxito.')
    else:
        print(f'Error en la exportación: {export_task.status()}')

Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'creation_timestamp_ms': 1725456762927, 'update_timestamp_ms': 1725456788030, 'start_timestamp_ms': 1725456767881, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'Y722BF6ZPFPZBZ3XBADE623W', 'name': 'projects/ee-facuboladerasgee/operations/Y722BF6ZPFPZBZ3XBADE623W'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'creation_timestamp_ms': 1725456762927, 'update_timestamp_ms': 1725456818030, 'start_timestamp_ms': 1725456767881, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'Y722BF6ZPFPZBZ3XBADE623W', 'name': 'projects/ee-facuboladerasgee/operations/Y722BF6ZPFPZBZ3XBADE623W'}
Exportación en progreso...
Error en la exportación: {'state': 'RUNNING', 'description': 'ExportSampleToCSV', 'creation_timestamp_ms': 1725456762927, 'update_timestamp_ms': 1725456848036, 'start_timestamp_ms': 1725456767881, 'task_type': 'EX